## Modeling with CNN for Wildlife Image Classification

The data for this project is sourced from https://www.kaggle.com/datasets/akash2907/bird-species-classification 

### 1. Import Packages and Load Data

In [1]:
# Import necessary modules
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import imageio.v2 as imageio
import cv2


In [28]:
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [35]:
# set path and folder names
train_path = "../jpeg/train"
test_path = "../jpeg/test"

folder_names = os.listdir(train_path)[1:]

In [18]:
# load images

# Create lists to store image data and labels
image_data=[]
image_labels = []

for folder in folder_names:
    img_path = os.path.join(train_path, folder) 
    
    for filename in os.listdir(img_path):
        if filename.endswith((".jpg",".JPG")):
            img = imageio.imread(os.path.join(img_path, filename))
            img = cv2.resize(img, (300, 300)) #resize
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY ) #convert to grayscale

            image_data.append(img)
            image_labels.append(folder)

# Convert data to numpy array
image_data = np.array(image_data)
print("success")

success


In [19]:
print('Shape of x_train array:', x_train.shape)
print('Shape of x_train labels:', len(image_labels))

Shape of x_train array: (149, 300, 300)
Shape of x_train labels: 149


#### Get the label mappings
The labels dictionary matches class names against the label indices used for training the model.

In [27]:
# Create a dictionary to map image names to their corresponding integer labels
name_to_label = {name: label for label, name in enumerate(folder_names)}

# Create a list to store the integer labels for each image
integer_labels = []

# Loop through the list of image names and assign the corresponding integer label
for name in image_labels:
    label = name_to_label.get(name)
    if label is not None:
        integer_labels.append(label)
    else:
        # Handle the case when the image name is not found in the mapping
        # Here, we'll set it to -1 to indicate an unknown label
        integer_labels.append(-1)

# Print the list of integer labels
#print(integer_labels)
print(len(integer_labels))

149


### Build base model
First, we need to one-hot-encode the target variable. 

In [29]:
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical


In [33]:
# Convert labels to one-hot encoded vectors
num_classes = 16  
one_hot_labels = to_categorical(integer_labels, num_classes=num_classes)

# Print the one-hot encoded labels
print(one_hot_labels)
one_hot_labels.shape

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


(149, 16)

### 2. Load Data and Apply Augmentations

Loading the training dataset and applying augmentations using ImageDataGenerator.

In [50]:
# Set the image and batch size
image_size = (200, 200)
batch_size = 32

# Number of classes
num_classes = 16


#### i. Base model -- create a model with very little augmentation or preprocessing

In [51]:
# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,      # Normalize pixel values to [0, 1]
    validation_split=0.2    # Split the data into 80% for training and 20% for validation
)

# Load and preprocess images from the directory
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Set to 'categorical' for one-hot encoded labels
    subset='training'          # Use the training subset of your data
)

validation_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Set to 'categorical' for one-hot encoded labels
    subset='validation'        # Use the validation subset of your data
)


Found 123 images belonging to 16 classes.
Found 26 images belonging to 16 classes.


In [52]:
# Create the model
model = Sequential()

# Add a 2D Convolutional layer
model.add(Conv2D(32, kernel_size=(5, 5), padding='valid', activation='relu', input_shape=(image_size[0], image_size[1], 3)))

# Add a MaxPooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add more Conv2D and MaxPooling2D layers as needed later ...

# Flatten the output from Convolutional layers
model.add(Flatten())

# Add a fully connected Dense layer
model.add(Dense(128, activation='relu'))

# Add the final Dense layer with softmax activation for multi-class classification
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model architecture
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 196, 196, 32)      2432      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 98, 98, 32)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 307328)            0         
                                                                 
 dense_4 (Dense)             (None, 128)               39338112  
                                                                 
 dense_5 (Dense)             (None, 16)                2064      
                                                                 
Total params: 39342608 (150.08 MB)
Trainable params: 39342608 (150.08 MB)
Non-trainable params: 0 (0.00 Byte)
__________

In [53]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/20
3/3 [==============================] - 12s 5s/step - loss: 25.6424 - accuracy: 0.0989
Epoch 2/20
3/3 [==============================] - 10s 3s/step - loss: 32.1674 - accuracy: 0.0833
Epoch 3/20
3/3 [==============================] - 9s 3s/step - loss: 17.4685 - accuracy: 0.1250
Epoch 4/20
3/3 [==============================] - 9s 3s/step - loss: 7.9461 - accuracy: 0.2637
Epoch 5/20
3/3 [==============================] - 9s 3s/step - loss: 3.6334 - accuracy: 0.2083
Epoch 6/20
3/3 [==============================] - 9s 3s/step - loss: 2.1651 - accuracy: 0.3407
Epoch 7/20
3/3 [==============================] - 9s 3s/step - loss: 2.0049 - accuracy: 0.5934
Epoch 8/20
3/3 [==============================] - 9s 3s/step - loss: 1.8109 - accuracy: 0.6374
Epoch 9/20
3/3 [==============================] - 9s 3s/step - loss: 1.5833 - accuracy: 0.6923
Epoch 10/20
3/3 [==============================] - 9s 3s/step - loss: 1.2123 - accuracy: 0.8242
Epoch 11/20
3/3 [===========================

After 18 epochs, the model accuracy is 1.00. This is a small image dataset, and this suggests that the model is likely overfitting the data. We'll try out data augmentation next and see how the model does. 

#### ii. Model 2 -- create a model with using augmentation and preprocessing

In [56]:
# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,      # Normalize pixel values to [0, 1]
    rotation_range=20,      # Randomly rotate images within the range of 20 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally within 10% of the image width
    height_shift_range=0.1, # Randomly shift images vertically within 10% of the image height
    horizontal_flip=True,   # Randomly flip images horizontally
    validation_split=0.2    # Split the data into 80% for training and 20% for validation
)

# Load and preprocess images from the directory
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Set to 'categorical' for one-hot encoded labels
    subset='training'          # Use the training subset of your data
)

validation_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Set to 'categorical' for one-hot encoded labels
    subset='validation'        # Use the validation subset of your data
)


Found 123 images belonging to 16 classes.
Found 26 images belonging to 16 classes.


In [57]:
## Use the same sequential model, but this time with augmented data

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model architecture
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 196, 196, 32)      2432      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 98, 98, 32)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 307328)            0         
                                                                 
 dense_4 (Dense)             (None, 128)               39338112  
                                                                 
 dense_5 (Dense)             (None, 16)                2064      
                                                                 
Total params: 39342608 (150.08 MB)
Trainable params: 39342608 (150.08 MB)
Non-trainable params: 0 (0.00 Byte)
__________

In [58]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/20
3/3 [==============================] - 12s 4s/step - loss: 5.3383 - accuracy: 0.2857
Epoch 2/20
3/3 [==============================] - 10s 3s/step - loss: 2.7251 - accuracy: 0.4167
Epoch 3/20
3/3 [==============================] - 10s 3s/step - loss: 2.3464 - accuracy: 0.2917
Epoch 4/20
3/3 [==============================] - 9s 3s/step - loss: 2.2734 - accuracy: 0.3407
Epoch 5/20
3/3 [==============================] - 9s 3s/step - loss: 2.1476 - accuracy: 0.3187
Epoch 6/20
3/3 [==============================] - 9s 3s/step - loss: 2.0796 - accuracy: 0.2857
Epoch 7/20
3/3 [==============================] - 9s 3s/step - loss: 1.9277 - accuracy: 0.3846
Epoch 8/20
3/3 [==============================] - 10s 4s/step - loss: 1.8404 - accuracy: 0.4945
Epoch 9/20
3/3 [==============================] - 9s 3s/step - loss: 1.6918 - accuracy: 0.5275
Epoch 10/20
3/3 [==============================] - 10s 3s/step - loss: 1.6516 - accuracy: 0.5385
Epoch 11/20
3/3 [===========================